In [20]:
import open3d
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import copy

import utils.registration as registration
import utils.fread as fread
import utils.pointcloud as pointcloud
import utils.grid_search as grid_search
import utils.functions as functions

from time import sleep
from PIL import Image
from utils.config import Config
from utils.depth_camera import DepthCamera

In [6]:
config = Config(
        sequence_dir="data/raw_data",
        feature_dir="data/features",
        output_dir="data/trajectories/trajectory/IMU_PCD",
        experiment="exp_12",
        trial="trial_1",
        subject="subject-1",
        sequence="01",
        groundtruth_dir="data/trajectories/groundtruth",
    )
    
config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [7]:
sequence_dir = config.get_sequence_dir()
feature_dir = config.get_feature_dir()

In [8]:
sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")

groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
groundtruth_t = groundtruth_data["local_t"]

In [11]:
depth_camera = DepthCamera("secondary", "data/raw_data/exp_12/metadata/device-3-aligned.json")

In [19]:
local_pcds = []

for i in tqdm.trange(len(sequence_ts)):
    if np.sum(groundtruth_t[i]) == 0:
        continue
    
    depth_image_file = os.path.join(sequence_dir, f"frame-{sequence_ts[i]}.depth.png")
    pcd = depth_camera.depth_to_point_cloud(depth_image_file)
    pcd.transform(groundtruth_t[i])
    
    local_pcds.append(pcd)

100%|██████████| 651/651 [00:20<00:00, 31.20it/s]


In [21]:
x = pointcloud.merge_pcds(local_pcds, 0.03)

In [ ]:
pointcloud.view(x)